In [1]:
# 🔍 DATA DIAGNOSTIC TOOL
import os

print("Checking for data on the Remote Server...")

# 1. Count actual images
image_count = 0
found_files = []

# Walk through current directory and /content to find ANY jpg/png
search_dirs = ['.', '/content']
for search_dir in search_dirs:
    if os.path.exists(search_dir):
        for root, dirs, files in os.walk(search_dir):
            for file in files:
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    image_count += 1
                    if len(found_files) < 3:
                        found_files.append(os.path.join(root, file))

print(f"\n📊 Total Images Found on Server: {image_count}")

if image_count == 0:
    print("❌ PROBLEM DETECTED: No images found on this server.")
    print("\n💡 EXPLANATION:")
    print("   You are running this notebook on a Remote GPU Server (Linux).")
    print("   But your files are likely on your Local Computer (Windows).")
    print("   The server cannot see your local files automatically.")
    print("\n🚀 SOLUTION:")
    print("   1. Zip your 'raw' folder on your computer.")
    print("   2. Drag and drop the 'raw.zip' file into the file list on the left (in VS Code or Colab interface).")
    print("   3. Run the next cell to unzip it.")
    
    data_root = None
else:
    print("✅ Images found! attempting to locate root folder...")
    # Attempt to derive data_root from the first found file
    # e.g. /content/data/raw/Non Demented/img1.jpg -> /content/data/raw
    first_img = found_files[0]
    # Go up two levels
    parent = os.path.dirname(first_img) # Non Demented
    grandparent = os.path.dirname(parent) # raw
    data_root = grandparent
    print(f"📂 Derived Data Root: {data_root}")


Checking for data on the Remote Server...

📊 Total Images Found on Server: 0
❌ PROBLEM DETECTED: No images found on this server.

💡 EXPLANATION:
   You are running this notebook on a Remote GPU Server (Linux).
   But your files are likely on your Local Computer (Windows).
   The server cannot see your local files automatically.

🚀 SOLUTION:
   1. Zip your 'raw' folder on your computer.
   2. Drag and drop the 'raw.zip' file into the file list on the left (in VS Code or Colab interface).
   3. Run the next cell to unzip it.


In [ ]:
# 🛠️ SMART UNZIP TOOL
import zipfile
import os

# Possible locations where raw.zip might be suitable
potential_zips = [
    'raw.zip', 
    'notebooks/raw.zip',
    '../raw.zip',
    '/content/raw.zip',
    '/content/notebooks/raw.zip'
]

found_zip = None
print("🔍 Looking for raw.zip...")
for p in potential_zips:
    if os.path.exists(p):
        found_zip = p
        print(f"✅ Found zip file at: {found_zip}")
        break

if found_zip:
    # Determine extraction path. If we are in 'notebooks', extract to '../data/raw' or 'data/raw' depending on structure
    # Safest bet: Extract to a known absolute data dir or relative 'data/raw'
    extract_to = 'data/raw'
    
    # If the zip is in 'notebooks/', we might want to extract to '../data/raw' if we are in project root
    # BUT, let's stick to current directory 'data/raw' to be safe and use that as root.
    
    print(f"📦 Extracting to '{extract_to}'...")
    os.makedirs(extract_to, exist_ok=True)
    
    with zipfile.ZipFile(found_zip, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        
    print("✅ Extraction Complete.")
    
    # Verify and Set Root
    if os.path.exists(os.path.join(extract_to, 'Data')):
        data_root = os.path.join(extract_to, 'Data')
        print(f"📂 Data Root set to inner folder: {data_root}")
    elif os.path.exists(os.path.join(extract_to, 'Non Demented')):
        data_root = extract_to
        print(f"📂 Data Root set to: {data_root}")
    elif os.path.exists(os.path.join(extract_to, 'raw', 'Data')):
        data_root = os.path.join(extract_to, 'raw', 'Data')
        print(f"📂 Data Root set to nested: {data_root}")
    else:
        # Just list what we have
        print(f"⚠️ Extracted, but check structure. Contents of {extract_to}: {os.listdir(extract_to)}")
        data_root = extract_to
else:
    print("❌ 'raw.zip' NOT FOUND.")
    print(f"   Checked CWD: {os.getcwd()}")


🔍 Looking for raw.zip...
❌ 'raw.zip' NOT FOUND.
   Checked CWD: /content


In [2]:
!pip install -q timm==0.9.12 einops==0.7.0 albumentations==1.3.1 PyYAML==6.0.2 "numpy<2.0" matplotlib seaborn scikit-learn pandas
import os
os.makedirs('data/raw', exist_ok=True)
os.makedirs('checkpoints', exist_ok=True)
os.makedirs('logs', exist_ok=True)
print("✅ Setup complete")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
✅ Setup complete


In [3]:
import os, yaml, warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
from einops import rearrange

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print(f"PyTorch {torch.__version__}, CUDA: {torch.cuda.is_available()}")


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
config = {
    'data': {
        'root_dir': data_root,
        'img_size': 224,
        'num_classes': 4,
        'class_names': [
            'Non Demented',
            'Very mild Dementia',
            'Mild Dementia',
            'Moderate Dementia'
        ],
        'train_split': 0.7,
        'val_split': 0.15,
        'test_split': 0.15
    },
    'model': {
        'cnn_backbone': 'efficientnet_b3',
        'pretrained': True,
        'vit_dim': 512,
        'vit_depth': 6,
        'vit_heads': 8,
        'vit_mlp_dim': 2048,
        'dropout': 0.2
    },
    'training': {
        'batch_size': 32,
        'epochs': 100,
        'learning_rate': 1e-4,
        'weight_decay': 1e-4,
        'label_smoothing': 0.1,
        'class_weights': [1.0, 2.0, 2.5, 3.0],
        'early_stopping_patience': 15,
        'use_amp': True,
        'grad_clip': 1.0
    },
    'paths': {
        'checkpoint_dir': './checkpoints',
        'best_model': './checkpoints/best_model.pth'
    }
}

print("✅ Config created")


✅ Config created


In [ ]:
class AlzheimerDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        image = Image.open(self.paths[idx]).convert("RGB")
        image = np.array(image)
        if self.transform:
            image = self.transform(image=image)["image"]
        return image, self.labels[idx]


In [ ]:
def get_transforms(train=True):
    if train:
        return A.Compose([
            A.Resize(224, 224),
            A.HorizontalFlip(p=0.5),
            A.Rotate(limit=10, p=0.5),
            A.Normalize(mean=[0.485]*3, std=[0.229]*3),
            ToTensorV2()
        ])
def get_transforms(train=True):
    if train:
        return A.Compose([
            A.Resize(224, 224),
            A.HorizontalFlip(p=0.5),
            A.Rotate(limit=10, p=0.5),
            A.Normalize(mean=[0.485]*3, std=[0.229]*3),
            ToTensorV2()
        ])
    return A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=[0.485]*3, std=[0.229]*3),
        ToTensorV2()
    ])

    return A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=[0.485]*3, std=[0.229]*3),
        ToTensorV2()
    ])


In [ ]:
def prepare_data(config):
    root = config['data']['root_dir']
    classes = config['data']['class_names']

    # Check if root directory exists
    if not os.path.exists(root):
        # Try alternative paths for Colab
        possible_paths = [
            './data/raw',
            '/content/data/raw',
            'data/raw',
        ]
        for path in possible_paths:
            if os.path.exists(path):
                root = path
                print(f"✅ Found data at: {root}")
                break
        else:
            raise FileNotFoundError(f"❌ Cannot find data directory. Tried: {possible_paths}")
    
    print(f"\n📂 Using data directory: {os.path.abspath(root)}")
    print(f"📂 Contents: {os.listdir(root)}\n")
    
    image_paths, labels = [], []

    print("📂 Scanning dataset:")
    for idx, cls in enumerate(classes):
        cls_dir = os.path.join(root, cls)
        
        # Debug: show what we're looking for
        if not os.path.isdir(cls_dir):
            print(f"❌ Missing: {cls_dir}")
            print(f"   Available folders: {os.listdir(root)}")
            raise FileNotFoundError(f"❌ Missing folder: {cls_dir}")

        imgs = [
            os.path.join(cls_dir, f)
            for f in os.listdir(cls_dir)
            if f.lower().endswith(('.jpg', '.png', '.jpeg'))
        ]

        print(f"  ✅ {cls}: {len(imgs)} images")
        image_paths.extend(imgs)
        labels.extend([idx] * len(imgs))

    image_paths = np.array(image_paths)
    labels = np.array(labels)

    X_temp, X_test, y_temp, y_test = train_test_split(
        image_paths, labels, test_size=0.15, stratify=labels, random_state=42
    )

    val_ratio = 0.15 / (0.7 + 0.15)
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=val_ratio, stratify=y_temp, random_state=42
    )

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)


(X_train, y_train), (X_val, y_val), (X_test, y_test) = prepare_data(config)
print(f"\n✅ Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")


📂 Using data directory: C:\Users\nisha\OneDrive\Documents\alzheimer-detection\data\raw\Data
📂 Contents: ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']

📂 Scanning dataset:
  ✅ Non Demented: 67222 images
  ✅ Very mild Dementia: 13725 images
  ✅ Mild Dementia: 5002 images
  ✅ Moderate Dementia: 488 images

✅ Train: 60505, Val: 12966, Test: 12966


In [ ]:
train_loader = DataLoader(
    AlzheimerDataset(X_train, y_train, get_transforms(True)),
    batch_size=32, shuffle=True, num_workers=2
)

val_loader = DataLoader(
    AlzheimerDataset(X_val, y_val, get_transforms(False)),
    batch_size=32, shuffle=False, num_workers=2
)

test_loader = DataLoader(
    AlzheimerDataset(X_test, y_test, get_transforms(False)),
    batch_size=32, shuffle=False, num_workers=2
)

print(f"✅ Dataloaders ready: {len(train_loader)} train batches")

✅ Dataloaders ready: 1891 train batches


In [ ]:
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels, embed_dim):
        super().__init__()
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=1)
        
    def forward(self, x):
        x = self.proj(x)
        x = rearrange(x, 'b c h w -> b (h w) c')
        return x

class MultiHeadAttention(nn.Module):
    def __init__(self, dim, heads=8, dropout=0.1):
        super().__init__()
        self.heads = heads
        self.scale = (dim // heads) ** -0.5
        self.qkv = nn.Linear(dim, dim * 3, bias=False)
        self.attn_drop = nn.Dropout(dropout)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(dropout)
        
    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.heads, C // self.heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

class TransformerBlock(nn.Module):
    def __init__(self, dim, heads, mlp_dim, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = MultiHeadAttention(dim, heads, dropout)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, mlp_dim), nn.GELU(), nn.Dropout(dropout),
            nn.Linear(mlp_dim, dim), nn.Dropout(dropout)
        )
        
    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

class VisionTransformer(nn.Module):
    def __init__(self, dim, depth, heads, mlp_dim, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([TransformerBlock(dim, heads, mlp_dim, dropout) for _ in range(depth)])
        self.norm = nn.LayerNorm(dim)
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return self.norm(x)

class HybridCNNViT(nn.Module):
    def __init__(self, num_classes=4, cnn_backbone='efficientnet_b3', pretrained=True,
                 vit_dim=512, vit_depth=6, vit_heads=8, vit_mlp_dim=2048, dropout=0.2):
        super().__init__()
        
        self.backbone = timm.create_model(cnn_backbone, pretrained=pretrained, features_only=True, out_indices=[-1])
        
        dummy = torch.randn(1, 3, 224, 224)
        with torch.no_grad():
            features = self.backbone(dummy)
            cnn_out_channels = features[-1].shape[1]
            feature_size = features[-1].shape[2]
        
        self.patch_embed = PatchEmbedding(cnn_out_channels, vit_dim)
        num_patches = feature_size * feature_size
        self.pos_embed = nn.Parameter(torch.randn(1, num_patches + 1, vit_dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, vit_dim))
        self.transformer = VisionTransformer(vit_dim, vit_depth, vit_heads, vit_mlp_dim, dropout)
        self.head = nn.Sequential(
            nn.LayerNorm(vit_dim), nn.Linear(vit_dim, vit_dim // 2), nn.GELU(),
            nn.Dropout(dropout), nn.Linear(vit_dim // 2, num_classes)
        )
        
    def forward(self, x):
        features = self.backbone(x)[-1]
        x = self.patch_embed(features)
        B, N, _ = x.shape
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat([cls_tokens, x], dim=1)
        x = x + self.pos_embed
        x = self.transformer(x)
        cls_output = x[:, 0]
        logits = self.head(cls_output)
        return logits

print("✅ Model architecture defined")

✅ Model architecture defined


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 Using device: {device}")

model = HybridCNNViT(
    num_classes=config['data']['num_classes'],
    cnn_backbone=config['model']['cnn_backbone'],
    pretrained=config['model']['pretrained'],
    vit_dim=config['model']['vit_dim'],
    vit_depth=config['model']['vit_depth'],
    vit_heads=config['model']['vit_heads'],
    vit_mlp_dim=config['model']['vit_mlp_dim'],
    dropout=config['model']['dropout']
).to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n📊 Model Summary:")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Model size: ~{total_params * 4 / (1024**2):.2f} MB")

# Test forward pass
test_input = torch.randn(2, 3, 224, 224).to(device)
with torch.no_grad():
    test_output = model(test_input)
print(f"\n✅ Model initialized successfully!")
print(f"   Input: {test_input.shape} → Output: {test_output.shape}")

🔥 Using device: cuda

📊 Model Summary:
   Total parameters: 29,366,060
   Trainable parameters: 29,366,060
   Model size: ~112.02 MB

✅ Model initialized successfully!
   Input: torch.Size([2, 3, 224, 224]) → Output: torch.Size([2, 4])


In [ ]:
# Loss function with class weights for imbalanced dataset
class_weights = torch.tensor(config['training']['class_weights']).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=config['training']['label_smoothing'])

# Optimizer
optimizer = optim.AdamW(
    model.parameters(),
    lr=config['training']['learning_rate'],
    weight_decay=config['training']['weight_decay']
)

# Learning rate scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=config['training']['epochs'],
    eta_min=1e-6
)

# Mixed precision scaler
scaler = GradScaler() if config['training']['use_amp'] else None

print("✅ Training components initialized")
print(f"   Loss: CrossEntropyLoss with class weights")
print(f"   Optimizer: AdamW (lr={config['training']['learning_rate']})")
print(f"   Scheduler: CosineAnnealingLR")
print(f"   Mixed Precision: {config['training']['use_amp']}")

✅ Training components initialized
   Loss: CrossEntropyLoss with class weights
   Optimizer: AdamW (lr=0.0001)
   Scheduler: CosineAnnealingLR
   Mixed Precision: True


In [ ]:
def train_epoch(model, loader, criterion, optimizer, device, scaler=None):
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []
    
    pbar = tqdm(loader, desc='Training', leave=False)
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        if scaler:
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), config['training']['grad_clip'])
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), config['training']['grad_clip'])
            optimizer.step()
        
        running_loss += loss.item()
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = accuracy_score(all_labels, all_preds)
    return epoch_loss, epoch_acc

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation', leave=False)
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            preds = outputs.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = accuracy_score(all_labels, all_preds)
    return epoch_loss, epoch_acc, all_preds, all_labels

print("✅ Training functions defined")

✅ Training functions defined


In [ ]:
os.makedirs(config['paths']['checkpoint_dir'], exist_ok=True)

history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
best_val_acc = 0.0
patience_counter = 0

print("\n" + "="*70)
print("🚀 STARTING TRAINING")
print("="*70)
print(f"Device: {device}")
print(f"Total epochs: {config['training']['epochs']}")
print(f"Batch size: {config['training']['batch_size']}")
print(f"Training samples: {len(X_train)}")
print("="*70 + "\n")

for epoch in range(config['training']['epochs']):
    print(f"\n📅 Epoch {epoch+1}/{config['training']['epochs']}")
    print("-" * 70)
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, scaler)
    
    # Validate
    val_loss, val_acc, val_preds, val_labels = validate(model, val_loader, criterion, device)
    
    # Update scheduler
    scheduler.step()
    
    # Store history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Print metrics
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
            'config': config
        }, config['paths']['best_model'])
        print(f"✅ Best model saved! Val Acc: {val_acc*100:.2f}%")
        patience_counter = 0
    else:
        patience_counter += 1
        print(f"⏳ Patience: {patience_counter}/{config['training']['early_stopping_patience']}")
    
    # Early stopping
    if patience_counter >= config['training']['early_stopping_patience']:
        print(f"\n⚠️ Early stopping triggered at epoch {epoch+1}")
        break

print(f"\n🎉 Training completed! Best Val Acc: {best_val_acc*100:.2f}%")

NameError: name 'os' is not defined